<a href="https://colab.research.google.com/github/StoneMason495/cap-comp215/blob/main/215Project2ZM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Daisy World: Testing the Gaia Hypothesis
###By: Zach Mason, COMP 215

In [2]:
import numpy as np
import matplotlib as plt
import random
from dataclasses import dataclass

##Daisy Class:
I created a simple data class to create each daisy, which only has its colour (inputted as a string), and the albedo, or amount of light reflected by a surface in decimal/fraction form (inputted as a float)
I chose the albedo values of 0.25 for black and 0.75 for white, as [this website](https://serc.carleton.edu/NAGTWorkshops/complexsystems/activities/daisyworld.html) utilized those values as well. The 0.75 value means that the white daisies will reflect 75% of the light shone upon them, where as the black daisies (with an albedo of 0.25) will absorb 75% of the light, and therefore the heat

In [3]:
B = lambda T: 1 - 0.003265*(22.5-T)**2 #The rate of growth for the daisies, T is equal to the temperature on the planet (if T<5 or T>40, then B = 0)
@dataclass
class Daisy: #a dataclass, with only the colour of the daisy, and the albedo value (fractional value of light that a surface reflects)
  color : str
  albedo : float
  pop : float
  
  def growth_rate(self, temp):
    growth_rate = self.pop * B(temp)
    return growth_rate if growth_rate > 0 else 0
  def death_rate(self, gamma=0.3):
    if self.pop > 0.01:
      return self.pop * gamma
    else:
      return 0
  def step(self):
    return 1

white = Daisy('white', 0.75, 0.02)
black = Daisy('black', 0.25, 0.02)

white.growth_rate(31), black.growth_rate(22.5)
white.death_rate(), black.death_rate()

(0.006, 0.006)

##Daisyworld Class:
To-do:


*   Add documentation
*   Remove Temp constant, make adaptable




In [4]:
class Daisyworld:
  def __init__(self, b_daisy, w_daisy, p=1.0):
    '''initialize a Daisyworld object, in preparation for a simulation
    b_daisy : Daisy
    w_daisy : Daisy
    p : float'''
    self.black = b_daisy #the black daisy used in the simulation, designed to be a daisy class object
    self.white = w_daisy #the white daisy used in the simulation, also designed to be a daisy class object
    self.p = p #the total planet percentage as a decimal, designed to be 1.0, but could in theory be lowered
    self.x = self.p - self.black.pop - self.white.pop #total planet remaining for expansion, utilized in calculations of growth
    self.luminosity = 0.5 #initial value of luminosity, which will then increase each step by (0.02*)
    self.sol_constant = 917
    self.stefboltz = 5.67*10**-8
    self.q = 0.2*(self.sol_constant/self.stefboltz)
    self.cycle_num = 0
  
  def dw_temp(self):
    '''calculates an the temperature for the year on Daisyworld, currently a constant of 30 degrees C'''
    # daisyworld__temp = ((solar_flux*(1-planetary_albedo)/stefboltz)^(0.25))-273
    temp = (((self.luminosity * self.sol_constant)*(1-self.dw_albedo())/self.stefboltz)**(0.25))-273
    return temp

  def dw_albedo(self):
    '''calculates the albedo (reflective quantity as a decimal/fraction) of the entire planet'''
    planet_albedo = (self.white.pop * self.white.albedo) + (self.black.pop * self.black.albedo) + (self.x*0.5)
    return planet_albedo

  def daisy_temp(self, daisy):
    '''Calculate the temperature of one of either black or white daisies in a daisyworld simulation'''
    # BD_temp = ((q*(planetary_albedo-BD_albedo)+((daisyworld__temp+273)^4))^(0.25))-273
    d_temp = ((self.q*(self.dw_albedo() - daisy.albedo)+((self.dw_temp() + 273)**4)))
    return 1

  def step(self):
    planet_temp = self.dw_temp()
    self.black.pop = self.black.pop + (self.black.growth_rate(planet_temp)*self.x)-self.black.death_rate()
    self.white.pop = self.white.pop + (self.white.growth_rate(planet_temp)*self.x)-self.white.death_rate()
    self.x = self.p - self.black.pop - self.white.pop
    planet_albedo = self.dw_albedo()
    self.cycle_num += 1
    self.luminosity += (0.02*self.cycle_num) if self.luminosity < 1 else 0

world = Daisyworld(black, white)
print(world.black)
print(world.dw_temp())
for i in range(10):
  world.step()
world.black, world.luminosity, world.cycle_num, world.dw_temp()

Daisy(color='black', albedo=0.25, pop=0.02)
-20.836878827969798


(Daisy(color='black', albedo=0.25, pop=0.06732554371841565),
 1.06,
 10,
 31.27448539463535)